In [1]:
print("Hello")

Hello


In [ ]:
# pip install datasets

In [2]:
import boto3 #

In [3]:
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

llm-finetune-dataset-ashfaq
llm-model-artifacts-sayed


In [4]:
s3_buckets = boto3.client("s3")

In [5]:
response = s3_buckets.list_objects_v2(Bucket= "llm-finetune-dataset-ashfaq",Prefix= 'datasets/')
for obj in response.get("Contents", []):
    print(obj)

{'Key': 'datasets/', 'LastModified': datetime.datetime(2025, 11, 25, 0, 56, 11, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'datasets/customer_support_responses_train.csv', 'LastModified': datetime.datetime(2025, 11, 27, 7, 18, 7, tzinfo=tzlocal()), 'ETag': '"bb726ada7f7f9189f5507f18d4c24904"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 281974, 'StorageClass': 'STANDARD'}
{'Key': 'datasets/pharma_instruction_data.csv', 'LastModified': datetime.datetime(2025, 11, 25, 0, 56, 46, tzinfo=tzlocal()), 'ETag': '"7a3bbfb7acae4d8dc88dc1a08501d745"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 2469, 'StorageClass': 'STANDARD'}


In [6]:
for obj in response.get("Contents", []):
    print(obj['Key'])

datasets/
datasets/customer_support_responses_train.csv
datasets/pharma_instruction_data.csv


## Training the model

### Loading the dataset

In [7]:
from datasets import load_dataset

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [8]:
# load the dataset
dataset_path = 's3://llm-finetune-dataset-ashfaq/datasets/customer_support_responses_train.csv'

In [9]:
dataset= load_dataset("csv", data_files= {'train': dataset_path}, split= 'train')
dataset

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:298: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'response'],
    num_rows: 3000
})

In [11]:
data_path = "s3://llm-finetune-dataset-ashfaq/datasets/pharma_instruction_data.csv"
dataset_pharma = load_dataset("csv", data_files= {'train': data_path}, split= 'train')
dataset_pharma

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})

In [ ]:
print(dataset[0])

- Creating a new column with combined instruction and response

In [15]:
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n### Response:\n{example['response']}"
    return {"text": prompt}

dataset= dataset.map(format_example)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [16]:
dataset

Dataset({
    features: ['instruction', 'response', 'text'],
    num_rows: 3000
})

In [17]:
dataset['text'][0]

'### Instruction:\nCustomer wants to cancel an order #1\n### Response:\nYou can cancel your order from your account dashboard within 24 hours.'

In [19]:
# pip install transformers peft

In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType


In [21]:
## getting the model from the hugging face

model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [23]:
tokenizer.eos_token

'</s>'

In [26]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [27]:
def tokenize_fn(example):
    tokens = tokenizer(example['text'], truncation= True, padding = "max_length", max_length=512)
    tokens['labels'] = tokens['input_ids'].copy()
    return tokens

In [28]:
tokenized = dataset.map(tokenize_fn, batched = True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [29]:
tokenized # text, input_ids, attention_mask, labels will be used to train the model

Dataset({
    features: ['instruction', 'response', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
})

In [32]:
model = AutoModelForCasualLM.from_pretrained

NameError: name 'AutoModelForCasualLM' is not defined